# Hybrid-RAG

# All Required Libearies

#### now 1st we will install all the required libraries

In [1]:
!pip -q install langchain huggingface_hub pinecone pinecone_client tiktoken
!pip -q install -U unstructured pdfminer.six pymupdf sentence_transformers
!pip -q install pytesseract pdf2image Pillow
!apt-get install -y tesseract-ocr libtesseract-dev poppler-utils
!pip -q install pytesseract langchain_community
!pip -q install chromadb faiss-cpu
!pip -q install langchain-google-genai
!pip -q install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 100.3 MB/s eta 0:00:00
  

### Before starting the project 1st we will ensure that the drive is mounted or not and also we are using "cuda" as a device for the faster outcome

## importing the required Libraries

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import Pinecone
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#### Now load the document that you want for the Context

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
path = "/content/drive/MyDrive/Internship projects/Sample Set/docs1"
document = PyPDFDirectoryLoader(path)
data = document.load()

In [16]:
len(data)

451

#### Text Splitting

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 512 , chunk_overlap = 150, length_function = len , is_separator_regex = False)
chunks = [splitter.split_text(doc.page_content) for doc in data]
flat_chunks = [chunk for sublist in chunks for chunk in sublist]


In [18]:
splitter1 = RecursiveCharacterTextSplitter(chunk_size=800,
                                          chunk_overlap=100)
chunk = splitter1.split_documents(data)

In [19]:
import pandas as pd

df = pd.DataFrame(flat_chunks , columns = ['vectors'])
df

,vectors
0,June 2023 GM1039E1-6 Edition 1\nFUJIFILM Busin...
1,2 1 Preface\n1 Preface\nThis reference guide...
2,"Refer\n Refer to ""Logging in to Authenticatio..."
3,can be acquired from our official website.\nht...
4,32.1 Administrator Information\n2 Setup\n2.1...
...,...
1537,44 5 Appendix
1538,455.2 Safety Notes\nSymbols Marked on the Ma...
1539,DO NOT THROW \nWASTE TONER \nCONTAINER INTO \n...
1540,CURLED PAPER\nDO NOT USE INK \nJET PRINTER \nP...


#### Loading the HuggingFace Embedding Model for creating an Embeddings of chunks that we created Earlier

#### This Well known Embedding Model:"BGE Model" Which is availbale on HuggingFace

In [20]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {'normalize_embeddings' : True}

embeddings = HuggingFaceBgeEmbeddings(
    model_name = model_name ,
    encode_kwargs= model_kwargs,
    model_kwargs = {'device':'cuda'},
)

#### Pinecone VectorStore setup (Implemented using Pinecone API-Key)

### Now setting up the another retriever "Chroma"

In [21]:
vectorstore = Chroma.from_documents(chunk, embeddings)

In [22]:
from langchain.vectorstores import FAISS
vectorstore1 = FAISS.from_documents(chunk , embeddings)

#### In HyBrid-RAG we use Ensamble Retreiver which requires Two diffrent retreivers and then it provides output based on the weights we assign to both retreivers

In [23]:
retriever1 = vectorstore.as_retriever(search_kwargs={"k": 3})
retriever2 = vectorstore1.as_retriever(search_kwargs={"k": 3})


##### so here is the Setup of Pinecone_vecotrstore retreiver and Chromadb_vectorStore retreiver

In [24]:
from langchain.retrievers import EnsembleRetriever
ensambel_reteriever = EnsembleRetriever(
    retrievers = [retriever1 , retriever2],
    weight = [0.6,0.4]

)

# Gemini LLM setup

In [25]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key = os.environ["GOOGLE_API_KEY"])

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI( model = 'gemini-1.5-flash' , temperature= 0.7)

# Prompt Template Designing

In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """
You are a knowledgeable assistant skilled in providing thorough and accurate responses. Use the provided context to answer the user's question comprehensively. Ensure that your answer covers all relevant aspects and is presented in a clear and organized manner and also try to give a full complete response and the response which is generated it should be visually reperesentable okk .
Context:
{context}

Question:
{query}

Answer (detailed, with examples if relevant):
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , template),
        ("human" , '{input}')
    ]
)

output_parser = StrOutputParser()

In [28]:
from langchain.schema.runnable import RunnableMap

In [29]:
import logging
logging.getLogger("langchain_community.vectorstores.pinecone").setLevel(logging.ERROR)

# Creating A QA Chain using Langchain and combining Prompt , Context and then it provided to LLM

In [30]:
chain = RunnableMap({
    "context": lambda x: ensambel_reteriever.get_relevant_documents(x["query"]),
    "query": lambda x: x["query"],
    "input": lambda x: x["query"]
}) | prompt | llm | output_parser

# Prepare the input data
input_data = {
    "query": "tell me how many notations are there in a maunal"
}

# Invoke the chain
result = chain.invoke(input_data)
print(result)

<ipython-input-30-0c36955fca08>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  "context": lambda x: ensambel_reteriever.get_relevant_documents(x["query"]),


Based on the provided text, there are at least 80 notations listed across pages 17, 18, and 19 of the manual.  These notations cover font names (e.g., Letter Gothic Italic [28], Arial Bold [37], New Century Schoolbook Roman [71]) and character sets/encodings (e.g., ISO 8859-1 Latin 1 [1], PC-850 [8], Windows 3.1 Latin 2 [13]).  The exact total might be higher if other pages of the manual contain additional notations.  The provided excerpts only show a portion of the manual's content.



In [31]:
input_data = {
    "query":"can you tell me what the things i should do Before Using the Machine"
}
result = chain.invoke(input_data)
result

'Before using the machine, you should:\n\n1. **Check the Power:** Confirm that the Reset button is in the reset position (pushed in) and that the power plug is securely connected.  If the power doesn\'t switch on after this, further troubleshooting may be necessary.\n\n2. **Wait for Warm-up:** If "Please wait..." is displayed on the control panel, the machine is warming up.  Do not attempt to use the machine during this time.\n\n3. **Consult the Reference Guide:** For detailed information about the control panel and its functions, refer to the "Reference Guide - Operations" document.  This guide will provide a comprehensive overview of the machine\'s controls and settings.\n\n4. **Address Condensation (if applicable):** If condensation has formed inside the machine, set the Sleep Mode trigger time to more than 60 minutes and let the machine run for 60 minutes to allow it to dry. Ensure all water droplets are removed from rollers and metal parts before use.  If condensation is frequent,

Before using the machine, you should:

1. **Check the Power:** Confirm that the Reset button is in the reset position (pushed in) and that the power plug is securely connected.  If the power doesn't switch on after this, further troubleshooting may be necessary.

2. **Wait for Warm-up:** If "Please wait..." is displayed on the control panel, the machine is warming up.  Do not attempt to use the machine during this time.

3. **Consult the Reference Guide:** For detailed information about the control panel and its functions, refer to the "Reference Guide - Operations" document.  This guide will provide a comprehensive overview of the machine's controls and settings.

4. **Address Condensation (if applicable):** If condensation has formed inside the machine, set the Sleep Mode trigger time to more than 60 minutes and let the machine run for 60 minutes to allow it to dry. Ensure all water droplets are removed from rollers and metal parts before use.  If condensation is frequent, consider keeping the Warmer Mode switched on via the control panel ([Device]>[System Settings]>[System Clock/Timers]>[Warmer Mode]>[On]).


These steps ensure the machine is properly prepared for use and help prevent potential issues.


In [32]:
chain.invoke({"query":" tell me step by step process of Paper Tray Settings "})

"The provided text describes the paper tray setup process in multiple steps, spread across different documents.  Here's a consolidated, step-by-step guide:\n\n\n**I. Loading Paper:**\n\n1. **Access the Paper Tray:** Pull out the paper tray towards you until it stops. (Ref: `rg_optional_2.pdf`, page 4)\n\n2. **Clear Existing Paper:** Remove any remaining paper from the tray. (Ref: `rg_optional_2.pdf`, page 4)\n\n3. **Adjust Paper Size Guides (If Necessary):**\n    * **Removing Guides:** If changing paper size, open the end guide, remove the screws securing the guides (front and back), and remove the guides from the tray. (Ref: `rg_optional_2.pdf`, page 4)\n    * **Inserting Guides:** Insert the guide protrusions into the appropriate paper size holes.  Reattach the guides to the tray and tighten the screws. (Ref: `rg_optional_2.pdf`, page 4)\n    * **Alternative Adjustment (Optional Units):** For optional units, move the lever at the top of the long-side paper guide to the rightmost posi

The provided text describes the paper tray setup process in multiple steps, spread across different documents.  Here's a consolidated, step-by-step guide:


**I. Loading Paper:**

1. **Access the Paper Tray:** Pull out the paper tray towards you until it stops. (Ref: `rg_optional_2.pdf`, page 4)

2. **Clear Existing Paper:** Remove any remaining paper from the tray. (Ref: `rg_optional_2.pdf`, page 4)

3. **Adjust Paper Size Guides (If Necessary):**
    * **Removing Guides:** If changing paper size, open the end guide, remove the screws securing the guides (front and back), and remove the guides from the tray. (Ref: `rg_optional_2.pdf`, page 4)
    * **Inserting Guides:** Insert the guide protrusions into the appropriate paper size holes.  Reattach the guides to the tray and tighten the screws. (Ref: `rg_optional_2.pdf`, page 4)
    * **Alternative Adjustment (Optional Units):** For optional units, move the lever at the top of the long-side paper guide to the rightmost position (<I> position) to enable changing.  Reset by moving it to the leftmost position. If accuracy isn't improved, move the lever at the right front to the rearmost position (<I> position). Reset by moving to the foremost position. After use, return the lever to the normal position <o>. (Ref: `rg_optional_2.pdf`, page 15)


4. **Load Paper:** Load the paper, ensuring it doesn't exceed the maximum fill line to prevent jams. (Ref: `rg_optional_2.pdf`, page 15)  Images are printed on the face side. (Ref: `rg_optional_2.pdf`, pages 4 and 15)

5. **Push in the Tray:** Push the paper tray back into the machine. (Ref: `rg_optional_2.pdf`, page 15)


**II. Configuring Paper Tray Settings (on the Home Screen):**

1. **Access Settings:**  After loading paper, go to the Home screen. (Ref: `rg_optional_2.pdf`, page 4)

2. **Open Paper Tray Settings:** The exact method to access the paper tray settings is not explicitly detailed in the provided text.  It implies a menu or button on the Home Screen.

3. **Configure Settings:** You'll now configure the following settings (Ref: `rg_operations_4.pdf`, page 19):

    * **[Paper Size]:**
        * **[Auto Detect]:** Let the machine automatically detect the paper size.
        * **[Manual Size Input]:** Manually enter the paper size.

    * **[Paper Type]:** Select the correct paper type.

    * **[Paper Color]:** Select the paper color.

    * **[Auto Tray Switching]:** Enable or disable automatic tray switching based on user-specified paper or when the current tray runs out of paper.  This setting requires Administrator Mode. (Ref: `rg_operations_4.pdf`, page 19)

    * **[Priority] (If Auto Tray Switching is enabled):** Set the priority for this tray if multiple trays are available. This also requires Administrator Mode. (Ref: `rg_operations_4.pdf`, page 19)

    * **[Auto Paper Select]:** Set conditions for automatic paper selection. This also requires Administrator Mode. (Ref: `rg_operations_4.pdf`, page 19)

4. **Save Settings:** Tap on [OK] to save the configured settings. (Ref: `rg_operations_4.pdf`, page 19)


**Important Notes:**

* **Administrator Mode:** Several settings require Administrator Mode access.  The provided text doesn't specify how to access this mode.
* **Specific Menu Navigation:** The exact location of the Paper Tray Settings menu within the machine's interface is not described.  Consult your machine's user manual for the precise steps.


This comprehensive guide combines information from the provided documents to give a complete picture of the paper tray setup process.  Remember to always refer to your specific machine's user manual for the most accurate and detailed instructions.
